# Colab / VS Code（Colab Kernel）初始化流程：讓你能成功執行 train.py

這份 Notebook 會從零開始：
1.（可選）確認你在 Colab GPU Runtime
2. 輸入 GitHub Token（若 repo 私有才需要）
3. 輸入 GitHub Repository 連結
4. Clone/Pull 專案
5. 安裝依賴
6. 執行 `train.py`

## 重要提醒（請務必看）
- **不要把 GitHub Token 明碼寫進 Notebook**，用互動輸入即可。
- 如果 repo 是 public，可以把 Token 留空。
- 在 Notebook 請盡量使用 `sys.executable -m pip` 安裝套件，避免 `pip` 和 `python` 指到不同環境。

## 1)（可選）確認你是不是 Colab Runtime（以及 GPU 是否可用）

- 若你是在 VS Code 透過 `google.colab` extension 連到 Colab Kernel：這些檢查會顯示 Colab 的路徑（常見是 `/content`）。
- 若你想用 GPU：請在 Colab 那邊把 Runtime 設定成 GPU（不同連線方式 UI 位置可能不同）。

In [20]:
import os, sys, platform, subprocess
print('Platform:', platform.platform())
print('Python:', sys.version)
print('Executable:', sys.executable)
print('CWD:', os.getcwd())

# 1) 先用系統層級的方式檢查：這台 runtime 有沒有 NVIDIA GPU / 驅動工具
print('\n[GPU check] nvidia-smi:')
try:
    out = subprocess.run(['bash', '-lc', 'command -v nvidia-smi && nvidia-smi -L'], capture_output=True, text=True)
    if out.returncode == 0 and out.stdout.strip():
        print(out.stdout.strip())
    else:
        # 常見情況：CPU runtime 會顯示找不到 nvidia-smi
        msg = (out.stderr or out.stdout or '').strip()
        print('(not available)')
        if msg:
            print(msg)
except Exception as e:
    print('(not available)', e)

# 2) 再用 PyTorch 檢查：torch 是否為 CUDA 版本、以及 CUDA 是否可用
print('\n[PyTorch check]')
try:
    import torch
    print('torch:', torch.__version__)
    print('torch.version.cuda:', torch.version.cuda)
    print('CUDA available:', torch.cuda.is_available())
    print('CUDA device count:', torch.cuda.device_count())
    if torch.cuda.is_available():
        print('GPU:', torch.cuda.get_device_name(0))
    else:
        print('\n結論：目前是 CPU 訓練環境（CUDA 不可用）。')
        print('原因通常是以下其中之一：')
        print('  A) 你連到的是「CPU runtime」：nvidia-smi 不存在（你剛剛看到的狀況）')
        print('  B) runtime 有 GPU，但你裝到的是 CPU-only torch（torch 版本會出現 +cpu，且 torch.version.cuda 會是 None）')
        print('\n要使用 GPU：需要在 Colab 端把 runtime 切成 GPU，並重新連線/重啟後再跑一次此檢查。')
        print('注意：修改這段檢查程式碼本身，不會把 CPU runtime 變成 GPU runtime。')
except Exception as e:
    print('torch not available yet (OK):', e)

Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Executable: /usr/bin/python3
CWD: /content/colab_model_Train_project

[GPU check] nvidia-smi:
(not available)

[PyTorch check]
torch: 2.9.0+cpu
torch.version.cuda: None
CUDA available: False
CUDA device count: 0

結論：目前是 CPU 訓練環境（CUDA 不可用）。
原因通常是以下其中之一：
  A) 你連到的是「CPU runtime」：nvidia-smi 不存在（你剛剛看到的狀況）
  B) runtime 有 GPU，但你裝到的是 CPU-only torch（torch 版本會出現 +cpu，且 torch.version.cuda 會是 None）

要使用 GPU：需要在 Colab 端把 runtime 切成 GPU，並重新連線/重啟後再跑一次此檢查。
注意：修改這段檢查程式碼本身，不會把 CPU runtime 變成 GPU runtime。


## 2) 輸入 GitHub Token 與 Repository 資訊

- Repo 是 **private**：需要 Token 才能 clone/pull
- Repo 是 **public**：Token 可留空（直接按 Enter）

這裡用 `GIT_USERNAME` + `GIT_REPO` 來組出 GitHub 連結：
- `https://github.com/<username>/<repo>.git`

In [4]:
# 建議不要把 GitHub Token 明碼寫進 notebook（容易外流）
# 1) 先在系統環境變數設定 GIT_TOKEN，或 2) 執行時再貼上
import os
import getpass
from pathlib import Path

# Public repo：可直接按 Enter 跳過 Token
GIT_TOKEN = os.getenv("GIT_TOKEN") or getpass.getpass("GitHub Token（repo public 可直接按 Enter 跳過，輸入不會顯示）: ")

# 你的 GitHub 帳號與 repo 名稱（可依需要修改）
GIT_USERNAME = "SelinaYeh2386"
GIT_REPO = "colab_model_Train_project"

# repo 資料夾路徑（會 clone 到目前工作目錄底下）
repo_dir = Path.cwd() / GIT_REPO
print("Repo folder:", repo_dir)

Repo folder: /content/colab_model_Train_project


## 3) Clone（若已存在則 Pull）

這段會：
- 如果資料夾不存在：clone
- 如果資料夾已存在：pull

> 小提醒：使用 Token clone 時，Token 可能會被寫進 `.git/config` 的 remote URL（有外洩風險）。這裡會盡量避免把 Token 印在輸出上，但你仍應該使用 fine-grained token、最小權限、用完可撤銷。

In [5]:
from urllib.parse import urlparse
import subprocess

# 組出 repo URL（不含 token）
REPO_URL = f"https://github.com/{GIT_USERNAME}/{GIT_REPO}.git"

# 需要 token（private repo）時，才在 clone 階段用帶 token 的 URL
if GIT_TOKEN:
    CLONE_URL = f"https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPO}.git"
else:
    CLONE_URL = REPO_URL

if not repo_dir.exists():
    print("Cloning into:", repo_dir)
    # 接下來執行 clone 指令（注意：不會把 token 值印出來）
    subprocess.run(["git", "clone", CLONE_URL, str(repo_dir)], check=True)

    # 重要：把 origin URL 改回不含 token，避免 token 被寫進 .git/config
    subprocess.run(["git", "-C", str(repo_dir), "remote", "set-url", "origin", REPO_URL], check=True)
else:
    print("Repo already exists, pulling latest:", repo_dir)
    subprocess.run(["git", "-C", str(repo_dir), "rev-parse", "--is-inside-work-tree"], check=True, capture_output=True, text=True)
    subprocess.run(["git", "-C", str(repo_dir), "pull"], check=True)

print("Done. Repo path:", repo_dir)

Repo already exists, pulling latest: /content/colab_model_Train_project
Done. Repo path: /content/colab_model_Train_project


## 4) 進入專案根目錄，確認 train.py / requirements.txt

如果你跑到這一步仍找不到 `train.py` 或 `requirements.txt`，代表 repo 可能多包了一層資料夾；你可以用 `find` 去找檔案位置。

In [6]:
import os
from pathlib import Path

os.chdir(repo_dir)
print('CWD:', Path.cwd())
print('Top-level files:', [p.name for p in sorted(Path.cwd().iterdir())][:50])

print('train.py exists:', Path('train.py').exists())
print('requirements.txt exists:', Path('requirements.txt').exists())
print('data.yaml exists:', Path('data.yaml').exists())

CWD: /content/colab_model_Train_project
Top-level files: ['.git', 'README.md', 'data.yaml', 'datasets', 'models', 'requirements.txt', 'train.py', 'yolov8n.pt']
train.py exists: True
requirements.txt exists: True
data.yaml exists: True


In [7]:
# 若 train.py 不在這層，用 find 找它
!find . -maxdepth 4 -name train.py

./train.py


In [8]:
# 若 requirements.txt 不在這層，用 find 找它
!find . -maxdepth 4 -name requirements.txt

./requirements.txt


## 5) 安裝依賴（建議用 sys.executable -m pip）

這一步會解決常見錯誤：
- `ModuleNotFoundError: No module named 'ultralytics'`

如果你的 `requirements.txt` 不在目前資料夾，請先 `cd` 到包含它的資料夾再安裝。

In [9]:
import sys
from pathlib import Path

print('Using Python:', sys.executable)
req = Path('requirements.txt')
if not req.exists():
    raise FileNotFoundError('requirements.txt 不在目前目錄，請先 cd 到正確位置')

!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install -r requirements.txt

Using Python: /usr/bin/python3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [10]:
# 驗證 ultralytics 是否可用
import ultralytics
from ultralytics import YOLO
print('ultralytics version:', ultralytics.__version__)

ultralytics version: 8.4.6


## 6) 執行訓練

- 建議同樣用 `sys.executable` 來執行 `train.py`，確保跟剛剛安裝套件的是同一個環境。
- 訓練結果預設會輸出到 `models/` 底下（例如 `models/my_exp/weights/best.pt`）。

In [19]:
import sys
!{sys.executable} train.py

Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=my_exp6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretr

In [12]:
# 找輸出權重檔
!find . -maxdepth 6 -name best.pt -o -name last.pt

./models/my_exp5/weights/last.pt
./models/my_exp5/weights/best.pt
./models/my_exp2/weights/last.pt
./models/my_exp2/weights/best.pt
./models/my_exp3/weights/last.pt
./models/my_exp3/weights/best.pt
./models/my_exp4/weights/last.pt
./models/my_exp4/weights/best.pt


## 6.5) 保存 / 下載訓練輸出（best.pt / last.pt）

你剛剛看到的 `models/**/weights/*.pt` **是存在 Colab runtime 的檔案**，不會自動跑回你本機的 VS Code 專案資料夾。

- 想把權重帶回本機：建議 **打包成 zip 後下載**（或改用雲端硬碟 / Release / LFS）。
- **不建議**把大型權重直接 `git commit` 進 repo（會很肥、也可能超過 GitHub 限制）。

In [ ]:
from pathlib import Path
import time
import zipfile

# 收集所有權重檔（依你的 train.py 設定，預設會在 models/**/weights/ 底下）
weight_files = sorted(Path('models').glob('**/weights/*.pt'))
print('Found weight files:', len(weight_files))
for p in weight_files[-10:]:
    print(' -', p)

if not weight_files:
    raise FileNotFoundError('找不到 models/**/weights/*.pt；請先完成訓練或確認輸出路徑')

# 以「最後修改時間」推測最新一次訓練的 run 資料夾
latest_weight = max(weight_files, key=lambda p: p.stat().st_mtime)
run_dir = latest_weight.parent.parent  # models/<exp_name>/weights/<file>.pt -> models/<exp_name>
print('Latest run dir:', run_dir)

# 打包：weights/*.pt + 常見的結果/設定檔（存在才加入）
candidates = []
candidates += sorted((run_dir / 'weights').glob('*.pt'))
for extra in ['args.yaml', 'results.csv', 'results.png']:
    p = run_dir / extra
    if p.exists():
        candidates.append(p)

zip_name = Path(f"{run_dir.name}_export_{time.strftime('%Y%m%d_%H%M%S')}.zip")
with zipfile.ZipFile(zip_name, 'w', compression=zipfile.ZIP_DEFLATED) as z:
    for p in candidates:
        # 壓到同一個根目錄：<exp_name>/...
        arcname = Path(run_dir.name) / p.relative_to(run_dir)
        z.write(p, arcname=str(arcname))

print('Created:', zip_name)
print('Absolute path:', zip_name.resolve())
print('Size (MB):', round(zip_name.stat().st_size / 1024 / 1024, 2))

# 若在 Colab 環境：嘗試觸發瀏覽器下載
try:
    from google.colab import files  # type: ignore
    files.download(str(zip_name))
except Exception as e:
    print('Auto-download not available in this environment (OK):', e)
    print('你可以用左側檔案瀏覽器手動下載該 zip，或把它移到你自己的雲端位置。')

## 7)（可選）把變更 commit + push 回 GitHub（建議只推程式碼/設定）

如果你在 Colab 端有改到程式碼（例如 `train.py`、`requirements.txt`、notebook），可以在這一步把變更 **commit + push** 回 GitHub。

> 注意：不建議把大型模型權重（例如 `models/**/weights/*.pt`）直接 commit 進 GitHub（檔案很大、也容易把 repo 搞到很慢）。通常會用 GitHub Release、雲端硬碟、或 Git LFS 來保存權重。

In [ ]:
import os
import getpass
import subprocess

# 需要 push 時，建議使用 token（private repo 必填；public repo 也可能需要）
if not GIT_TOKEN:
    GIT_TOKEN = os.getenv("GIT_TOKEN") or getpass.getpass("GitHub Token（用於 git push，輸入不會顯示）: ")

# 設定 git 身分（若已設定會略過）
subprocess.run(["git", "config", "--global", "user.name"], capture_output=True, text=True)
name = subprocess.run(["git", "config", "--global", "user.name"], capture_output=True, text=True).stdout.strip()
email = subprocess.run(["git", "config", "--global", "user.email"], capture_output=True, text=True).stdout.strip()
if not name:
    subprocess.run(["git", "config", "--global", "user.name", GIT_USERNAME], check=True)
if not email:
    subprocess.run(["git", "config", "--global", "user.email", f"{GIT_USERNAME}@users.noreply.github.com"], check=True)

# 暫時把 origin 設成含 token 的 URL（push 完會改回不含 token）
AUTH_URL = f"https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPO}.git"
subprocess.run(["git", "remote", "set-url", "origin", AUTH_URL], check=True)

# 顯示狀態
print(subprocess.run(["git", "status"], capture_output=True, text=True).stdout)

# 只要你確認要推回 GitHub，就 add/commit/push
subprocess.run(["git", "add", "-A"], check=True)
msg = input("Commit message（例如: update notebook / code）: ").strip() or "Update from Colab"
commit = subprocess.run(["git", "commit", "-m", msg], capture_output=True, text=True)
print(commit.stdout or commit.stderr)

# 如果沒有變更可提交，git commit 會回傳非 0；仍可嘗試 push（通常會顯示 up-to-date）
push = subprocess.run(["git", "push"], capture_output=True, text=True)
print(push.stdout or push.stderr)

# 把 origin 改回不含 token，避免 token 留在 .git/config
subprocess.run(["git", "remote", "set-url", "origin", REPO_URL], check=True)
print("origin reset to:", REPO_URL)

查詢Colab VM Repo下的檔案架構指令:

In [13]:
!find {repo_dir} -maxdepth 2 -type d -print

/content/colab_model_Train_project
/content/colab_model_Train_project/models
/content/colab_model_Train_project/models/my_exp5
/content/colab_model_Train_project/models/my_exp2
/content/colab_model_Train_project/models/my_exp3
/content/colab_model_Train_project/models/my_exp
/content/colab_model_Train_project/models/my_exp4
/content/colab_model_Train_project/.git
/content/colab_model_Train_project/.git/objects
/content/colab_model_Train_project/.git/branches
/content/colab_model_Train_project/.git/refs
/content/colab_model_Train_project/.git/logs
/content/colab_model_Train_project/.git/info
/content/colab_model_Train_project/.git/hooks
/content/colab_model_Train_project/datasets
/content/colab_model_Train_project/datasets/coco8


In [22]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [15]:
import torch; print(torch.__version__); print(torch.cuda.is_available()); print(torch.version.cuda)

2.9.0+cpu
False
None


In [17]:
!git commit

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@5c9642094d92.(none)')


In [18]:
!git config --global user.email "Selina.yeh@insyde.com"
!git config --global user.name "SelinaYeh2386"



In [23]:
!git commit

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	models/my_exp4/
	models/my_exp5/
	models/my_exp6/

nothing added to commit but untracked files present (use "git add" to track)


In [24]:
!git add .

In [ ]:
!git commit

<odel_Train_project/.git/COMMIT_EDITMSG" 60L, 2568B▽  Pzz\[0%m           [>c]10;?]11;?# Please enter the commit message for your changes. Lines starting# with '#' will be ignored, and an empty message aborts the commit.#
# On branch main
# Your branch is up to date with 'origin/main'.
#
# Changes to be committed:
#       new file:   models/my_exp4/BoxF1_curve.png
#       new file:   models/my_exp4/BoxPR_curve.png
#       new file:   models/my_exp4/BoxP_curve.png
#       new file:   models/my_exp4/BoxR_curve.png
#       new file:   models/my_exp4/args.yaml
#       new file:   models/my_exp4/confusion_matrix.png
#       new file:   models/my_exp4/confusion_matrix_normalized.png
#       new file:   models/my_exp4/labels.jpg
#       new file:   models/my_exp4/results.csv
#       new file:   models/my_exp4/results.png
#       new file:   models/my_exp4/train_batch0.jpg
#       new file:   models/my_exp4/train_batch1.jpg
#       new file:   models/my_exp4/train_batch2.jpg
#       new